# Curve Registry
Demonstration of using the Curve Registry to find and interact with Curve contracts

### Useful links:
* [`Source code`](https://github.com/curvefi/curve-pool-registry/): <https://github.com/curvefi/curve-pool-registry/>
- [`AddressProvider`](https://github.com/curvefi/curve-pool-registry/blob/master/contracts/AddressProvider.vy): [0x0000000022D53366457F9d5E68Ec105046FC4383](https://etherscan.io/address/0x0000000022d53366457f9d5e68ec105046fc4383)
- [`Registry`](https://github.com/curvefi/curve-pool-registry/blob/master/contracts/Registry.vy): [0x7D86446dDb609eD0F5f8684AcF30380a356b2B4c](https://etherscan.io/address/0x7D86446dDb609eD0F5f8684AcF30380a356b2B4c)
- [`PoolInfo`](https://github.com/curvefi/curve-pool-registry/blob/master/contracts/PoolInfo.vy): [0xe64608E223433E8a03a1DaaeFD8Cb638C14B552C](https://etherscan.io/address/0xe64608E223433E8a03a1DaaeFD8Cb638C14B552C)
- [`Swaps`](https://github.com/curvefi/curve-pool-registry/blob/master/contracts/Swaps.vy): [0xD1602F68CC7C4c7B59D686243EA35a9C73B0c6a2](https://etherscan.io/address/0xD1602F68CC7C4c7B59D686243EA35a9C73B0c6a2)


## Setup
Set up the environment by importing libraries, functions

In [ ]:
# Initialize Brownie environment
# <https://eth-brownie.readthedocs.io/en/stable/python-package.html#accessing-the-network>

from brownie import network, Contract
import json

#XXX Make sure your WEB3 INFURA PROJECT ID is set as an environment variable, i.e. 
# %env WEB3_INFURA_PROJECT_ID=YOUR KEY HERE

In [2]:
# Functions used throughout

# Load an ABI stored in a file
def load_abi(filename):
    with open(filename, 'r') as f:
        abi = json.load(f)
    return abi

# Load an ERC20 token
def load_coin(address, abi_file = 'erc20.abi'):
    return Contract.from_abi("ERC20", address, load_abi(abi_file))

In [3]:
# Human Readable Addresses used for testing

# Registry Addresses
provider_addr = '0x0000000022d53366457f9d5e68ec105046fc4383'

# Tokens
usdt_addr = '0xdAC17F958D2ee523a2206206994597C13D831ec7'
usdc_addr = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
dai_addr = '0x6B175474E89094C44Da98b954EedeAC495271d0F'

# Y pool is a commonly used Curve pool
# underlying coins: DAI/USDC/USDT/TUSD
# compounding coins: yDAI/yUSDC/yUSDT/yTUSD
y_pool_addr = '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51' 

# sUSD is a Curve metapool between 3pool and Synthetix USD
susd_pool_addr = '0xA5407eAE9Ba41422680e2e00537571bcC53efBfD'

# External DAI exchange to convert ETH to Dai
dai_exchange_addr = '0x2a1530C4C41db0B0b2bB646CB5Eb1A67b7158667'

# A return value of '0' for an address is 0x000...
zero_addr = '0x0000000000000000000000000000000000000000'

## Address Provider
The Address Provider links to the active Curve contracts.  This section shows how to retrieve the current Registry address

In [4]:
# Connect to the live Ethereum blockchain
network.connect('mainnet')

In [5]:
# The registry may change, the Address Provider contract will point to the current registry address

provider_contract = Contract.from_abi('NewAddressIdentifier', provider_addr, load_abi('provider.abi'))
registry_addr = provider_contract.get_registry()

print(f"The current registry address is {registry_addr}")

The current registry address is 0x7D86446dDb609eD0F5f8684AcF30380a356b2B4c


In [6]:
# Load the address of the Pool Info contract from the Address Provider

pool_info_addr = provider_contract.get_address(1) # The first offset is the Pool Info contract

print(f"The current address of the Pool Info contract is {pool_info_addr}")

The current address of the Pool Info contract is 0xe64608E223433E8a03a1DaaeFD8Cb638C14B552C


In [7]:
# Load the address of the Swaps contract from the Address Provider

swaps_addr = provider_contract.get_address(2) # The second offset is the address of the swaps contract

print(f"The current address of the Swaps contract is {swaps_addr}")

The current address of the Swaps contract is 0xD1602F68CC7C4c7B59D686243EA35a9C73B0c6a2


## Registry
From the registry contract, all Curve pools can be accessed.  This section shows how to interact with the registry on the live mainnet

In [8]:
# Load the registry
registry_contract = Contract.from_abi('CurveRegistry', registry_addr, load_abi('registry.abi'))

# Loop through current pools
pool_count = registry_contract.pool_count()
print(f"Curve currently has {pool_count} pools")

Curve currently has 32 pools


In [9]:
# Loop through first 3 pools.  Replace "3" with `pool_count` to loop all pools
for i in range(3):
    # The pool_list function pulls the pool address by index
    address = registry_contract.pool_list(i)
    print(f"\nPool {i + 1} is located at {address} consisting of:")
    
    # The get_coins function retrieves the first eight coins within a pool
    coins = registry_contract.get_coins(address)
    
    for c in coins:
        # get_coins retrieves 8 addresses, returning blank addresses if the pool has fewer than 8
        if c == zero_addr:
            break
            
        # Use our function defined above to interact with an ERC20 token
        coin_data = load_coin(c)
        print(f" {coin_data.symbol()} (${coin_data.name()}) located at {c}")


Pool 1 is located at 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 consisting of:
 DAI ($Dai Stablecoin) located at 0x6B175474E89094C44Da98b954EedeAC495271d0F
 USDC ($USD Coin) located at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
 USDT ($Tether USD) located at 0xdAC17F958D2ee523a2206206994597C13D831ec7

Pool 2 is located at 0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27 consisting of:
 yDAI ($iearn DAI) located at 0xC2cB1040220768554cf699b0d863A3cd4324ce32
 yUSDC ($iearn USDC) located at 0x26EA744E5B887E5205727f55dFBE8685e3b21951
 yUSDT ($iearn USDT) located at 0xE6354ed5bC4b393a5Aad09f21c46E101e692d447
 yBUSD ($iearn BUSD) located at 0x04bC0Ab673d88aE9dbC9DA2380cB6B79C4BCa9aE

Pool 3 is located at 0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56 consisting of:
 cDAI ($Compound Dai) located at 0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643
 cUSDC ($Compound USD Coin) located at 0x39AA39c021dfbaE8faC545936693aC917d5E7563


### Finding a pool by coins
Pools can be found for a given pair (from -> to). There is an array of pools per pair on chain, and you need to pass an integer (0, 1, 2...) to iterate over all pools in existence who have this currency pair.
Once there are no more pools existing for that pair, zero address is returned.

In [10]:
# Query the Registry contract to find the first pool that exchanges USDT for DAI
registry_contract.find_pool_for_coins(usdt_addr, dai_addr, 0)

'0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7'

In [11]:
# Query the Registry contract to find the second pool that exchanges USDT for DAI
registry_contract.find_pool_for_coins(usdt_addr, dai_addr, 1)

'0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27'

In [12]:
# If we check a very high offset (20) we should not expect to find a pool, but the zero address

pool_too_far = registry_contract.find_pool_for_coins(usdt_addr, dai_addr, 20)  # ... eventually we hit 0x0000
if pool_too_far == zero_addr:
    print("No pool found at this index")
else:
    print(f"Found {pool_too_far}")

No pool found at this index


In [13]:
# Rates are the effective rate of the coins.  Y pool has four coins
rates = registry_contract.get_rates(y_pool_addr)
for r in rates:
    if r != 0:
        print(r / 1e18)

1.0674601574371512
1.1968430112258905
1.0582256307261866
1.035785230609678


## Pool Info
The Pool Info contract is a convenient endpoint to retrieve multiple data points about Curve pools

In [14]:
# Pool Info provides a contract that loads many registry endpoints within one data structure
pool_info = Contract.from_abi('PoolInfo', pool_info_addr, load_abi('poolinfo.abi'))

Return Format:
```
     [
        balances: Registry.get_balances(),
        underlying_balances: Registry.get_underlying_balances(),
        decimals: Registry.get_decimals(),
        underlying_decimals: Registry).get_underlying_decimals(),
        rates: Registry.get_rates(),
        lp_token: Registry.get_lp_token(),
        params: Registry.get_parameters(),
      ]
```

With rates, underlying_balance = balance * rate / 1e18

The Parameters are a tuple containing:

``` 
(
    pool_params.A = CurvePool().A(),
    pool_params.future_A = CurvePool().future_A(),
    pool_params.fee = CurvePool().fee(),
    pool_params.future_fee = CurvePool().future_fee(),
    pool_params.admin_fee = CurvePool().admin_fee(),
    pool_params.future_admin_fee = CurvePool().future_admin_fee(),
    pool_params.future_owner = CurvePool().future_owner()
```
_Optionally contains the following if an initial_A value exists_
```
    pool_params.initial_A = CurvePool(_pool).initial_A(),  
    pool_params.initial_A_time = CurvePool(_pool).initial_A_time(),
    pool_params.future_A_time = CurvePool(_pool).future_A_time()
)
```

In [15]:
# Some example data from PoolInfo for Y Pool

y_info = pool_info.get_pool_info(y_pool_addr)
print(f"Info for the first coin in {y_pool_addr}")
print(f"Balance is {y_info[0][0] / (10 ** y_info[2][0])}")
print(f"Precision is {y_info[2][0]} Decimals")

print()
print("Other pool info:")
print(f"Current amplification factor is {y_info[6][0]}")
if y_info[6][0] == y_info[6][1]:
    print(f"This amplification factor is not scheduled to change")
else:
    print(f"This amplification factor is schedule to change to {y_info[6][1]}")
    

Info for the first coin in 0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51
Balance is 17212566.706174113
Precision is 18 Decimals

Other pool info:
Current amplification factor is 1000
This amplification factor is not scheduled to change


### Estimate the most probable gas usage for exchanging coins

In [16]:
# Estimate gas usage for exchanging coins
# XXX Does not work without calculator connected
calculator_connected = False

if calculator_connected:
    registry_contract.estimate_gas_used(y_pool_addr,
                           usdt_addr,
                           dai_addr)

### Calculate exchange amount(s)
We're swapping USDC (6 digits) to DAI (18 digits)

In [17]:
# Load the exchange contract from the address we got from the Registry
exchange_contract = Contract.from_abi('Exchange', swaps_addr, load_abi('exchange.abi'))

In [18]:
# What is exchange rate we can get for converting USDC to DAI?
dai_amount = exchange_contract.get_exchange_amount(
                             y_pool_addr,
                             usdc_addr,
                             dai_addr,
                             10 ** 6)  # Dump 1 USDC
dai_amount / 1e18  # DAI has 18 digits

0.9986814073279512

How much USDC do we need to get 1 DAI?

In [19]:
# IF we want to get a final amount of 1 DAI, how much USDC do we need to put in?
# XXX Does not work without calculator connected

if calculator_connected:
    usdc_amount = exchange_contract.get_input_amount(
                             y_pool_addr,
                             usdc_addr,
                             dai_addr,
                             10 ** 18)
    usdc_amount / 1e6

Get many exchange amounts (DAI for USDC) at once

In [20]:
# What are the exchange amounts for a range of values?
# XXX Does not work without calculator connected
if calculator_connected:
    amounts = exchange_contract.get_exchange_amounts(
                             y_pool_addr,
                             usdc_addr,
                             dai_addr,
                             [x * 10 ** 6 for x in range(1, 101)])
    [x / 1e18 for x in amounts][:5]  # Let's show only first 5 out of 100

In [21]:
# Disconnect from the network to prepare for the next section that deals with moving value around.
network.disconnect()

### Exchanges

Reconnect to a fork of mainnet on ganache-cli, giving local access to a live blockchain snapshot.

In [ ]:
# Connect directly to mainnet-fork
network.connect('mainnet-fork')

# Another method of connecting to a local forked mainnet:
# network.connect('development')

In [23]:
# Load the contracts for the ERC20 tokens we'll be interacting with
dai_contract = load_coin(dai_addr)
usdc_contract = load_coin(usdc_addr)

In [24]:
# Trade some Eth from default accounts for DAI

# Create "Alice" account from Brownie pre-populated addresses
from brownie import accounts
alice = accounts[0]

# Convert Alice's ETH to DAI using a DAI exchange
dai_exchange_contract = Contract.from_abi('DaiExchange', dai_exchange_addr, load_abi('dai_exchange.abi'))
dai_exchange_contract.ethToTokenSwapInput(1, 9999999999, {'from': alice, 'value': '10 ether'})

Transaction sent: 0x2767517ce4cd0ba32bbe6b7638d7044a42f8f89591f555595820bc079f0ef5d5
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 1
  DaiExchange.ethToTokenSwapInput confirmed - Block: 11934767   Gas used: 66972 (0.56%)



<Transaction '0x2767517ce4cd0ba32bbe6b7638d7044a42f8f89591f555595820bc079f0ef5d5'>

**Make ERC20 contract objects to check balances before and after transaction**

In [26]:
# Store initial balance and verify > 0
initial_dai_balance = dai_contract.balanceOf(alice) / 1e18  # DAI is 18 decimal places
print(f"Alice's initial balance is {initial_dai_balance}")

Alice's initial balance is 14377.72027796293


In [27]:
# Alice should have no USDC
usdc_contract.balanceOf(alice) / 1e6  # USDC is 6 decimal places

0.0

In [28]:
# Approve token transfer in the source contract
dai_contract.approve(exchange_contract, 5 * 1e18, {'from': alice}) # Approve 5 dollars

Transaction sent: 0x8f837086a9e42bf9b5a11f28aacae01f218c416a25f8e4c3e0a85d63a308f1ce
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 2
  ERC20.approve confirmed - Block: 11934768   Gas used: 44046 (0.37%)



<Transaction '0x8f837086a9e42bf9b5a11f28aacae01f218c416a25f8e4c3e0a85d63a308f1ce'>

In [29]:
# Perform the exchange
exchange_contract.exchange(
                  susd_pool_addr,                              # SUSD pool
                  dai_addr,                                    # from DAI
                  usdc_addr,                                   # to USDC
                  10 ** 18,                                    # swap 1 dollar
                  10 ** 6 // 2,                                # require no less than half a dollar
                  {'from': alice})

Transaction sent: 0xbeefc1c22852bc5268730f27c7dd7953f38a633cf0a42ea4798f74673b194237
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 3
  Exchange.exchange confirmed - Block: 11934769   Gas used: 193244 (1.61%)



<Transaction '0xbeefc1c22852bc5268730f27c7dd7953f38a633cf0a42ea4798f74673b194237'>

In [30]:
# Alice's balance has dropped
final_dai_balance = dai_contract.balanceOf(alice) / 1e18
print(f"Alice's balance has dropped from ${round(initial_dai_balance,0)} to ${round(final_dai_balance,0)}")
print(f"The exact difference is ${(initial_dai_balance - final_dai_balance)}")

Alice's balance has dropped from $14378.0 to $14377.0
The exact difference is $1.0


In [33]:
# Alice's final balance has increased
usdc_contract.balanceOf(alice) / 1e6

1.000538